In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torchvision.models import vgg16, VGG16_Weights
from torch.utils.data import DataLoader
from PIL import Image
import os

In [12]:
# 1. Data transformations

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

In [13]:
# 2. Load dataset
#take dataset from the pd then add the path here

train_loader = DataLoader(datasets.ImageFolder('dataset/train', transform=transform), batch_size=16, shuffle=True)
val_loader = DataLoader(datasets.ImageFolder('dataset/val', transform=transform), batch_size=16)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'dataset/train'

In [16]:
# 3. Load pre-trained VGG16

# Old: model = models.vgg16(pretrained=True)
model = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)

In [17]:
#4. Freeze convolutional layers

for param in model.features.parameters():
    param.requires_grad = False

In [18]:
# 5. Replace classifier for new classes

num_classes = len(train_loader.dataset.classes)
model.classifier = nn.Sequential(
    nn.Linear(25088, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, num_classes)
)

NameError: name 'train_loader' is not defined

In [20]:
# 6. Device, loss and optimizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

In [21]:
# 7. Training loop

for epoch in range(5):
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

NameError: name 'train_loader' is not defined

In [22]:
# 8. Validation accuracy

model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        _, preds = torch.max(model(images), 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
print(f'Validation Accuracy: {100 * correct / total:.2f}%')

NameError: name 'val_loader' is not defined

In [23]:
# 9. Test on single/multiple images
# ---------------------------
test_images = ["dataset/test/sample1.jpg", "dataset/test/sample2.jpg"]  # add paths of images to test
for img_path in test_images:
    img = Image.open(img_path).convert("RGB")
    img_t = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(img_t)
        _, predicted = torch.max(outputs, 1)
    classes = train_loader.dataset.classes
    print(f"Image: {os.path.basename(img_path)} -> Predicted class: {classes[predicted.item()]}")

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/test/sample1.jpg'

In [24]:
# Dummy image (3x224x224) for demonstration
dummy_img = torch.randn(1, 3, 224, 224).to(device)

with torch.no_grad():
    outputs = model(dummy_img)
    _, predicted = torch.max(outputs, 1)

classes = ["class1", "class2", "class3"]  # example class names
print(f"Predicted class for dummy image: {classes[predicted.item()]}")


IndexError: list index out of range